In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# 노트북 안에 그래프 그리기
%matplotlib inline

# ggplot: 그래프에서 격자로 숫자 범위 눈에 잘 띄도록 하는 스타일
plt.style.use('ggplot')

# 그래프에서 마이넛 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_test_ns_dq = pd.read_csv("/content/drive/MyDrive/24-2/데이터과학응용/Question3/question-3_test_ns_dq.csv", encoding='ISO-8859-1')
print(df_test_ns_dq.shape)
df_test_ns_dq.head()

(4869, 17)


,Unnamed: 0,id,Sex,Event,Equipment,Age,BodyweightKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,TotalKg,Place,Dots,Wilks,Glossbrenner,Goodlift
0,15,15,M,SBD,Raw,36.0,110.50,125.0,132.5,-135.0,132.5,NaN,DQ,NaN,NaN,NaN,NaN
1,18,18,M,SBD,Raw,36.0,101.70,135.0,-142.5,142.5,142.5,NaN,DQ,NaN,NaN,NaN,NaN
2,35,35,F,B,Single-ply,32.0,71.55,-105.0,-110.0,-110.0,NaN,NaN,DQ,NaN,NaN,NaN,NaN
3,51,51,M,B,Single-ply,38.0,82.15,-165.0,-165.0,-165.0,NaN,NaN,DQ,NaN,NaN,NaN,NaN
4,65,65,M,B,Single-ply,39.0,97.90,-155.0,-155.0,-155.0,NaN,NaN,DQ,NaN,NaN,NaN,NaN


In [4]:
df_test_final = pd.read_excel("/content/drive/MyDrive/24-2/데이터과학응용/Question3/question3-test - features.xlsx")
print(df_test_final.shape)
df_test_final.head()

(99006, 23)


,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,Bench1Kg,Bench2Kg,...,Place,Dots,Wilks,Glossbrenner,Goodlift,Federation,MeetCountry,MeetTown,MeetName,Sanctioned
0,M,SBD,Raw,NaN,NaN,NaN,Open,95.4,160.0,167.5,...,2,403.90,398.90,381.44,83.02,KNKF-SP,Netherlands,Tilburg,DRC Cup,Yes
1,M,SBD,Raw,NaN,NaN,NaN,Open,102.35,140.0,145.0,...,3,387.01,382.99,365.71,79.34,KNKF-SP,Netherlands,Tilburg,DRC Cup,Yes
2,M,SBD,Raw,28.5,24-34,24-39,Open,100.2,142.5,-147.5,...,4,384.37,380.07,362.99,78.88,KNKF-SP,Netherlands,Tilburg,DRC Cup,Yes
3,M,SBD,Raw,23.5,24-34,24-39,Open,99.35,120.0,-125.0,...,5,382.70,378.32,361.38,78.57,KNKF-SP,Netherlands,Tilburg,DRC Cup,Yes
4,M,SBD,Raw,NaN,NaN,NaN,Open,102.25,140.0,-145.0,...,6,378.02,374.08,357.20,77.5,KNKF-SP,Netherlands,Tilburg,DRC Cup,Yes


In [5]:
df_test = pd.read_csv("/content/drive/MyDrive/24-2/데이터과학응용/Question3/question-3_test_other_cleaned.csv", encoding='ISO-8859-1')
print(df_test.shape)
df_test.head()

(94137, 26)


,Unnamed: 0,id,Sex,Event,Equipment,Age,BodyweightKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Goodlift,Sex_label,Dots_coeff,Wilks_coeff,Glossbrenner_coeff,TotalKg_dots,TotalKg_wilks,TotalKg_glossbrenner,Event_num,squat_plus_dead
0,0,0,M,SBD,Raw,33.0,95.40,160.0,167.5,-172.5,...,83.02,1,0.628643,0.620853,0.593679,642.495107,642.503166,642.502143,3,475.000139
1,1,1,M,SBD,Raw,32.0,102.35,140.0,145.0,150.0,...,79.34,1,0.609460,0.603131,0.575915,635.004403,635.002953,635.006815,3,485.004723
2,2,2,M,SBD,Raw,28.0,100.20,142.5,-147.5,-147.5,...,78.88,1,0.614984,0.608105,0.580790,625.007765,625.007579,624.993391,3,482.502912
3,3,3,M,SBD,Raw,23.0,99.35,120.0,-125.0,125.0,...,78.57,1,0.617264,0.610190,0.582866,619.993751,620.003451,620.005652,3,495.000951
4,4,4,M,SBD,Raw,31.0,102.25,140.0,-145.0,-145.0,...,77.50,1,0.609710,0.603353,0.576130,619.999830,620.001731,619.998712,3,480.000091


In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/24-2/데이터과학응용/Question3/question-3_train_cleaned_final.csv", encoding='ISO-8859-1')
print(df_train.shape)
df_train.head()

(884452, 24)


,Unnamed: 0,Sex,Event,Equipment,Age,BodyweightKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,...,Goodlift,Sex_label,Dots_coeff,Wilks_coeff,Glossbrenner_coeff,TotalKg_dots,TotalKg_wilks,TotalKg_glossbrenner,Event_num,squat_plus_dead
0,0,F,B,Raw,43.0,73.2,80.0,85.0,90.0,90.0,...,70.06,0,0.986694,0.965490,0.850020,89.997481,89.995743,89.997909,1,0.0
1,1,F,B,Raw,26.0,60.6,40.0,42.5,45.0,45.0,...,38.42,0,1.101631,1.106341,0.977385,44.996906,45.004212,44.997607,1,0.0
2,2,F,B,Raw,19.0,50.3,32.5,35.0,-37.5,35.0,...,34.55,0,1.247692,1.278819,1.135188,35.000633,35.001039,34.998618,1,0.0
3,3,F,B,Raw,19.0,63.7,40.0,42.5,-45.0,42.5,...,35.25,0,1.068333,1.065043,0.940322,42.496120,42.495949,42.496087,1,0.0
4,4,M,B,Raw,30.0,82.0,127.5,132.5,0.0,132.5,...,66.97,1,0.679738,0.672363,0.647124,132.506854,132.502913,132.493987,1,0.0


In [7]:
features = ['Sex', 'Event', 'Equipment', 'Age', 'BodyweightKg','Best3BenchKg','Dots','Wilks', 'Glossbrenner','TotalKg_dots', 'TotalKg_wilks','TotalKg_glossbrenner', 'Event_num', 'squat_plus_dead']
target_col = "TotalKg"

X_train = df_train[features]
y_train = df_train[target_col]
X_test = df_test[features]
y_test = df_test[target_col]

In [8]:
categorical_features = ['Sex', 'Event', 'Equipment']
X_train_encoded = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)

In [9]:
X_train_encoded

,Age,BodyweightKg,Best3BenchKg,Dots,Wilks,Glossbrenner,TotalKg_dots,TotalKg_wilks,TotalKg_glossbrenner,Event_num,...,Event_D,Event_S,Event_SB,Event_SBD,Event_SD,Equipment_Raw,Equipment_Single-ply,Equipment_Straps,Equipment_Unlimited,Equipment_Wraps
0,43.0,73.20,90.0,88.80,86.89,76.50,89.997481,89.995743,89.997909,1,...,False,False,False,False,False,True,False,False,False,False
1,26.0,60.60,45.0,49.57,49.79,43.98,44.996906,45.004212,44.997607,1,...,False,False,False,False,False,True,False,False,False,False
2,19.0,50.30,35.0,43.67,44.76,39.73,35.000633,35.001039,34.998618,1,...,False,False,False,False,False,True,False,False,False,False
3,19.0,63.70,42.5,45.40,45.26,39.96,42.496120,42.495949,42.496087,1,...,False,False,False,False,False,True,False,False,False,False
4,30.0,82.00,132.5,90.07,89.09,85.74,132.506854,132.502913,132.493987,1,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884447,31.0,67.50,112.5,348.76,348.88,338.66,452.496248,452.493991,452.501555,3,...,False,False,False,True,False,True,False,False,False,False
884448,25.0,72.70,102.5,323.96,322.37,311.91,442.496405,442.494033,442.505915,3,...,False,False,False,True,False,True,False,False,False,False
884449,29.0,82.15,155.0,458.35,453.34,436.29,675.006629,674.994227,674.999589,3,...,False,False,False,True,False,True,False,False,False,False
884450,31.0,81.35,137.5,419.95,415.51,400.06,614.995844,614.995987,615.001075,3,...,False,False,False,True,False,True,False,False,False,False


In [28]:
X_test_strap = X_test[X_test['Equipment'] == 'Straps']
print(X_test_strap)

Empty DataFrame
Columns: [Sex, Event, Equipment, Age, BodyweightKg, Best3BenchKg, Dots, Wilks, Glossbrenner, TotalKg_dots, TotalKg_wilks, TotalKg_glossbrenner, Event_num, squat_plus_dead]
Index: []


In [29]:
X_train_strap = X_train[X_train['Equipment'] == 'Straps']
print(X_train_strap)

       Sex Event Equipment   Age  BodyweightKg  Best3BenchKg    Dots   Wilks  \
173741   M     D    Straps  25.0          88.0           0.0  228.93  226.07   
173742   M     D    Straps  42.0          97.6           0.0  236.41  233.59   
173743   M     D    Straps  28.0         109.2           0.0  213.81  212.34   
173744   M     D    Straps  39.0         115.3           0.0  222.70  222.10   
173745   M     D    Straps  38.0         121.1           0.0  223.30  223.75   
...     ..   ...       ...   ...           ...           ...     ...     ...   
302737   M     D    Straps  27.0          91.2           0.0  218.40  215.62   
302738   M     D    Straps  28.0          96.6           0.0  212.51  209.93   
302739   M     D    Straps  26.0          97.3           0.0  199.36  196.97   
302740   M     D    Straps  31.0         107.3           0.0  221.24  219.47   
302741   M     D    Straps  30.0         139.9           0.0  202.81  206.78   

        Glossbrenner  TotalKg_dots  Tot

In [10]:
X_test_encoded['Equipment_Straps'] = False
X_test_encoded['Sex_Mx'] = False

In [11]:
X_train_encoded.columns

Index(['Age', 'BodyweightKg', 'Best3BenchKg', 'Dots', 'Wilks', 'Glossbrenner',
       'TotalKg_dots', 'TotalKg_wilks', 'TotalKg_glossbrenner', 'Event_num',
       'squat_plus_dead', 'Sex_M', 'Sex_Mx', 'Event_BD', 'Event_D', 'Event_S',
       'Event_SB', 'Event_SBD', 'Event_SD', 'Equipment_Raw',
       'Equipment_Single-ply', 'Equipment_Straps', 'Equipment_Unlimited',
       'Equipment_Wraps'],
      dtype='object')

In [12]:
X_test_encoded.columns

Index(['Age', 'BodyweightKg', 'Best3BenchKg', 'Dots', 'Wilks', 'Glossbrenner',
       'TotalKg_dots', 'TotalKg_wilks', 'TotalKg_glossbrenner', 'Event_num',
       'squat_plus_dead', 'Sex_M', 'Event_BD', 'Event_D', 'Event_S',
       'Event_SB', 'Event_SBD', 'Event_SD', 'Equipment_Raw',
       'Equipment_Single-ply', 'Equipment_Unlimited', 'Equipment_Wraps',
       'Equipment_Straps', 'Sex_Mx'],
      dtype='object')

In [13]:
new_column_order = ['Age', 'BodyweightKg', 'Best3BenchKg', 'Dots', 'Wilks', 'Glossbrenner',
       'TotalKg_dots', 'TotalKg_wilks', 'TotalKg_glossbrenner', 'Event_num',
       'squat_plus_dead', 'Sex_M', 'Sex_Mx', 'Event_BD', 'Event_D', 'Event_S',
       'Event_SB', 'Event_SBD', 'Event_SD', 'Equipment_Raw',
       'Equipment_Single-ply', 'Equipment_Straps', 'Equipment_Unlimited',
       'Equipment_Wraps']
X_test_encoded = X_test_encoded[new_column_order]

In [14]:
X_test_encoded.columns

Index(['Age', 'BodyweightKg', 'Best3BenchKg', 'Dots', 'Wilks', 'Glossbrenner',
       'TotalKg_dots', 'TotalKg_wilks', 'TotalKg_glossbrenner', 'Event_num',
       'squat_plus_dead', 'Sex_M', 'Sex_Mx', 'Event_BD', 'Event_D', 'Event_S',
       'Event_SB', 'Event_SBD', 'Event_SD', 'Equipment_Raw',
       'Equipment_Single-ply', 'Equipment_Straps', 'Equipment_Unlimited',
       'Equipment_Wraps'],
      dtype='object')

In [15]:
rf_model = RandomForestRegressor(
    n_estimators=120,              # Small increase in trees
    max_depth=15,                  # Add a moderate maximum depth constraint for stability
    max_features=0.9,              # Reduce max features slightly to avoid overfitting
    random_state=42,               # Set random state for reproducibility
    ccp_alpha=0.001                # Small complexity penalty to slightly prune the tree
)
rf_model.fit(X_train_encoded, y_train)

y_pred = rf_model.predict(X_test_encoded)

print(y_pred)

[642.54365213 634.04483693 625.08667483 ... 371.27159186 369.74476065
 369.85575146]


In [16]:
len(y_pred)

94137

In [17]:
df_test['TotalKg'] = y_pred

df_test.head(20)

,Unnamed: 0,id,Sex,Event,Equipment,Age,BodyweightKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Goodlift,Sex_label,Dots_coeff,Wilks_coeff,Glossbrenner_coeff,TotalKg_dots,TotalKg_wilks,TotalKg_glossbrenner,Event_num,squat_plus_dead
0,0,0,M,SBD,Raw,33.0,95.40,160.0,167.5,-172.5,...,83.02,1,0.628643,0.620853,0.593679,642.495107,642.503166,642.502143,3,475.000139
1,1,1,M,SBD,Raw,32.0,102.35,140.0,145.0,150.0,...,79.34,1,0.609460,0.603131,0.575915,635.004403,635.002953,635.006815,3,485.004723
2,2,2,M,SBD,Raw,28.0,100.20,142.5,-147.5,-147.5,...,78.88,1,0.614984,0.608105,0.580790,625.007765,625.007579,624.993391,3,482.502912
3,3,3,M,SBD,Raw,23.0,99.35,120.0,-125.0,125.0,...,78.57,1,0.617264,0.610190,0.582866,619.993751,620.003451,620.005652,3,495.000951
4,4,4,M,SBD,Raw,31.0,102.25,140.0,-145.0,-145.0,...,77.50,1,0.609710,0.603353,0.576130,619.999830,620.001731,619.998712,3,480.000091
5,5,5,M,SBD,Raw,26.0,100.60,130.0,135.0,140.0,...,76.84,1,0.613931,0.607147,0.579843,610.003687,610.000429,609.992772,3,469.998963
6,6,6,M,SBD,Raw,27.0,104.05,135.0,140.0,-142.5,...,75.01,1,0.605324,0.599486,0.572424,604.998158,605.001277,605.006039,3,465.001825
7,7,7,M,SBD,Raw,36.0,94.70,150.0,0.0,0.0,...,77.80,1,0.630804,0.622922,0.595805,599.995934,599.995074,599.995240,3,449.995416
8,8,8,M,SBD,Raw,36.0,104.80,150.0,157.5,162.5,...,73.53,1,0.603561,0.597954,0.570981,595.002416,594.995144,594.993449,3,432.497003
9,9,9,M,SBD,Raw,35.0,102.10,145.0,155.0,160.0,...,73.18,1,0.610085,0.603688,0.576455,585.000043,585.004221,585.006597,3,425.003620


In [18]:
len(df_test)

94137

In [41]:
df_test.to_csv("question-3_submission.csv", index = False)

In [42]:
# TotalKg 열의 값이 0인 행을 필터링
zero_totalkg_rows = df_test[df_test['TotalKg'] == 0]

# 결과 출력
print(zero_totalkg_rows)


Empty DataFrame
Columns: [Unnamed: 0, id, Sex, Event, Equipment, Age, BodyweightKg, Bench1Kg, Bench2Kg, Bench3Kg, Best3BenchKg, TotalKg, Place, Dots, Wilks, Glossbrenner, Goodlift, Sex_label, Dots_coeff, Wilks_coeff, Glossbrenner_coeff, TotalKg_dots, TotalKg_wilks, TotalKg_glossbrenner, Event_num, squat_plus_dead]
Index: []

[0 rows x 26 columns]


In [43]:
# 1. df_test_ns_dq 데이터프레임의 'TotalKg' 열의 모든 값을 0으로 채우기
df_test_ns_dq['TotalKg'] = 0

In [44]:
df_test_ns_dq.head()

,Unnamed: 0,id,Sex,Event,Equipment,Age,BodyweightKg,Bench1Kg,Bench2Kg,Bench3Kg,Best3BenchKg,TotalKg,Place,Dots,Wilks,Glossbrenner,Goodlift
0,15,15,M,SBD,Raw,36.0,110.50,125.0,132.5,-135.0,132.5,0,DQ,NaN,NaN,NaN,NaN
1,18,18,M,SBD,Raw,36.0,101.70,135.0,-142.5,142.5,142.5,0,DQ,NaN,NaN,NaN,NaN
2,35,35,F,B,Single-ply,32.0,71.55,-105.0,-110.0,-110.0,NaN,0,DQ,NaN,NaN,NaN,NaN
3,51,51,M,B,Single-ply,38.0,82.15,-165.0,-165.0,-165.0,NaN,0,DQ,NaN,NaN,NaN,NaN
4,65,65,M,B,Single-ply,39.0,97.90,-155.0,-155.0,-155.0,NaN,0,DQ,NaN,NaN,NaN,NaN


In [45]:


# 2. df_test와 df_test_ns_dq를 합치기
combined_df = pd.concat([df_test, df_test_ns_dq])

# 'id' 열을 기준으로 오름차순으로 정렬
combined_df = combined_df.sort_values(by='id').reset_index(drop=True)

# 결과 출력
print(combined_df)


       Unnamed: 0     id Sex Event Equipment   Age  BodyweightKg  Bench1Kg  \
0               0      0   M   SBD       Raw  33.0         95.40     160.0   
1               1      1   M   SBD       Raw  32.0        102.35     140.0   
2               2      2   M   SBD       Raw  28.0        100.20     142.5   
3               3      3   M   SBD       Raw  23.0         99.35     120.0   
4               4      4   M   SBD       Raw  31.0        102.25     140.0   
...           ...    ...  ..   ...       ...   ...           ...       ...   
99001       99001  99001   F   SBD       Raw  20.0         57.95      60.0   
99002       99002  99002   F   SBD       Raw  22.0         61.56      60.0   
99003       99003  99003   F   SBD       Raw  22.0         61.87     -62.5   
99004       99004  99004   F   SBD       Raw  22.0         62.61      75.0   
99005       99005  99005   F   SBD       Raw  19.0         62.76      70.0   

       Bench2Kg  Bench3Kg  ...  Goodlift  Sex_label Dots_coeff 

In [46]:
combined_df.columns

Index(['Unnamed: 0', 'id', 'Sex', 'Event', 'Equipment', 'Age', 'BodyweightKg',
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Best3BenchKg', 'TotalKg', 'Place',
       'Dots', 'Wilks', 'Glossbrenner', 'Goodlift', 'Sex_label', 'Dots_coeff',
       'Wilks_coeff', 'Glossbrenner_coeff', 'TotalKg_dots', 'TotalKg_wilks',
       'TotalKg_glossbrenner', 'Event_num', 'squat_plus_dead'],
      dtype='object')

In [47]:
df_test_final.columns

Index(['Sex', 'Event', 'Equipment', 'Age', 'AgeClass', 'BirthYearClass',
       'Division', 'BodyweightKg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg',
       'Best3BenchKg', 'TotalKg', 'Place', 'Dots', 'Wilks', 'Glossbrenner',
       'Goodlift', 'Federation', 'MeetCountry', 'MeetTown', 'MeetName',
       'Sanctioned'],
      dtype='object')

In [50]:
# combined_df의 'TotalKg' 열을 df_final에 추가
df_test_final['TotalKg'] = combined_df['TotalKg'].values

# 결과 출력
print(df_test_final['TotalKg'])


0        642.50
1        635.00
2        625.00
3        620.00
4        620.00
          ...  
99001    377.50
99002    375.00
99003    372.65
99004    370.00
99005    370.00
Name: TotalKg, Length: 99006, dtype: float64


In [52]:
print(combined_df['TotalKg'])


0        642.50
1        635.00
2        625.00
3        620.00
4        620.00
          ...  
99001    377.50
99002    375.00
99003    372.65
99004    370.00
99005    370.00
Name: TotalKg, Length: 99006, dtype: float64


In [53]:
df_test.to_csv("question-3_submission_final.csv", index = False)